# Tests

In [1]:
import os
import sys
import logging
import datetime
import requests
import json
import re
import pandas as pd

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib.nanuq import Nanuq
from lib.gapp import Phenotips
from lib.gapp import BSSH
from lib.samplesheet import SampleSheet

nq    = Nanuq()
pho   = Phenotips()
bssh  = BSSH()

class Args:
    # Simulate an argparse instance
    #
    def __init__(self, run='20240510_LH00336_0043_A22K5KMLT3', level='info'):
        self.run   = run
        self.level = level

args = Args(level='debug')
vars(args)

{'run': '20240510_LH00336_0043_A22K5KMLT3', 'level': 'debug'}

In [4]:
pd.read_csv('D:\HSJ\Projects\PRAGMatIQ\Runs\LH00336_0043\samples_list.csv')

,sample_name,biosample,relation,gender,ep_label,mrn,cohort_type,status,family_id,birthdate,flowcell_date
0,GM241091,26418,PROBAND,FEMALE,CHUSJ,03446823,TRIO,AFF,03446823,2017-04-24,2024-05-10
1,GM241062,26416,MTH,FEMALE,CHUSJ,03479431,TRIO,UNF,03446823,1991-05-08,2024-05-10
2,GM241110,26419,FTH,MALE,CHUSJ,03479514,TRIO,UNF,03446823,1990-07-21,2024-05-10
3,GM241018,26414,PROBAND,MALE,CHUSJ,03471602,TRIO,AFF,03471602,2024-02-18,2024-05-10
4,GM241067,26417,MTH,FEMALE,CHUSJ,01581134,TRIO,UNF,03471602,1990-08-19,2024-05-10
5,GM241047,26415,FTH,MALE,CHUSJ,03479135,TRIO,UNF,03471602,1982-06-08,2024-05-10
6,24-04046-T1,26362,PROBAND,MALE,CHUQ,1753303 CHUL,TRIO,AFF,24-04046-T1,2024-04-23,2024-05-10
7,24-04048-T1,26363,MTH,FEMALE,CHUQ,1742355 CHUL,TRIO,UNK,24-04046-T1,2000-04-19,2024-05-10
8,24-04049-T1,26364,FTH,MALE,CHUQ,1729707 CHUL,TRIO,UNK,24-04046-T1,2000-05-17,2024-05-10
9,24-04052-T1,26370,PROBAND,MALE,CHUQ,1603220 CHUL,TRIO,AFF,24-04052-T1,2019-08-07,2024-05-10


In [9]:
biosample     = "25335"
biosamplename = "MO-24-003080"
biosampleid   = bssh.get_biosampleid(biosample)

In [13]:
datasets = bssh.get_datasets(biosampleid)
bssh.get_sequenced_files(20756)

d:\HSJ\Workspace\cqgc-dev\lib\gapp.py:484: UserWarning: WARNING: Found more than one dataset for 2275456
  warnings.warn(f"WARNING: Found more than one dataset for {biosampleid}")


['/projects/3703702/biosamples/2275456/datasets/ds.16ef9da98da745bf9f51c57e89c282f6/sequenced files/135667656',
 '/projects/3703702/biosamples/2275456/datasets/ds.16ef9da98da745bf9f51c57e89c282f6/sequenced files/135667657',
 '/projects/3703702/biosamples/2275456/datasets/ds.c9d163e092094f2b86a069af1c103bb7/sequenced files/135667644',
 '/projects/3703702/biosamples/2275456/datasets/ds.c9d163e092094f2b86a069af1c103bb7/sequenced files/135667645',
 '/projects/3703702/biosamples/2275456/datasets/ds.5da4765e9ef649a0bd0feb1afdf5390e/sequenced files/135667616',
 '/projects/3703702/biosamples/2275456/datasets/ds.5da4765e9ef649a0bd0feb1afdf5390e/sequenced files/135667617',
 '/projects/3703702/biosamples/2275456/datasets/ds.d671465658bf428b9db4fc6ba0b2bda4/sequenced files/135667614',
 '/projects/3703702/biosamples/2275456/datasets/ds.d671465658bf428b9db4fc6ba0b2bda4/sequenced files/135667615']

In [4]:
endpoint = '/v2/datasets/'
url      = bssh.server + endpoint
payload  = {'inputbiosamples': {biosampleid}} # , 'datasettypes': ''}
response = requests.get(url, headers=bssh.headers, params=payload)
response.raise_for_status

# item0 = response.json().get('Items')[0]
# print(f"{item0['Project']['Id']} {item0['Project']['Name']}")
# print(json.dumps(item0["Project"], indent=2))

<bound method Response.raise_for_status of <Response [200]>>

In [5]:
def get_datasets(self, biosampleid):
    """
    Get BSSH dataset ID, project ID and project Name for `biosampleid`.
    - `biosampleid`: [str], Id of biosample
    - Returns: [list] of tuples 
    """
    endpoint = '/v2/datasets/'
    url      = self.server + endpoint
    payload  = {'inputbiosamples': {biosampleid}} # , 'datasettypes': ''}
    # FastQ uploaded using CLI has DatasetTypes.ID 'illumina.fastq.v1.8' 
    # while the ones created by BCL Convert have the type 'common.fastq'.

    response = requests.get(url, headers=self.headers, params=payload)
    response.raise_for_status

    items  = response.json().get('Items')
    counts = response.json().get('Paging')['TotalCount']

    datasets = []
    if len(items) != counts:
        logging.warning(f"WARNING: Found {len(items)} datasets but expected {counts} for {biosampleid}")
    for item in items:
        datasets.append((item['Id'], item['Project']['Id'], item['Project']['Name']))
    if len(datasets) != 1:
        logging.warning(f"WARNING: Found more than one dataset for {biosampleid}")
    return datasets

In [6]:
datasets = get_datasets(bssh, biosampleid)
for dataset in datasets:
    datasetid, projectid, projectname = dataset
    print(f"{datasetid} {projectid} {projectname}")

ds.add451fb133443ca96c46b956102c1e7 5412410 PRAGMatIQ_CUSM


In [ ]:
content = nq.get_samplesheet('LH00336_0009').text.splitlines()
print(content[1:10])

In [ ]:
# Get authorization token from Emedgene
# Please note - the Authorization header is only valid for a limited time, and
# expires after 8H. In that case, any request made with an expired token will 
# return a 403 code. To resolve, re-do the Login procedure to get a new token.
#
url      = "https://chusaintejustine.emedgene.com/api/auth/api_login/"
payload  = '{"username": "cqgc.bioinfo.hsj@ssss.gouv.qc.ca", "password": "3175CoteSainte-Catherine"}'
headers  = {'Content-Type': 'application/json'}
response = requests.request("POST", url, headers=headers, data=payload)
auth_header = response.json()["Authorization"]
auth_header


In [ ]:
# Get case for EMG634551172 (GM240123)
#
case_id = "EMG634551172"
get_test_response = requests.get(f'https://chusaintejustine.emedgene.com/api/test/{case_id}/', json={}, headers={'Authorization': auth_header})
get_test_response.json().keys()


In [ ]:
# Transfer files to aws
# --profile "emedgene" or "emedgene-eval"
aws s3 ls --profile emedgene-eval s3://cac1-prodca-emg-downloads/Ste_Justine_eval/upload